In [1]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY
from datasets import load_dataset_builder

In [2]:
rg.server_info()

ServerInfo(url='http://localhost:6900', version='1.21.0-dev', elasticsearch_version='8.5.3')

In [8]:
# download dataset from huggingface
imdb = load_dataset("imdb")

Using the latest cached version of the module from /home/ankush/.cache/huggingface/modules/datasets_modules/datasets/imdb/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0 (last modified on Thu Dec  7 11:47:14 2023) since it couldn't be found locally at imdb., or remotely on the Hugging Face Hub.


[12/16/23 17:05:54] WARNING  WARNING:datasets.load:Using the latest cached version of the module from  ]8;id=169677;file:///home/ankush/anaconda3/envs/argilla_dev_env/lib/python3.9/site-packages/datasets/load.py\load.py]8;;\:]8;id=44717;file:///home/ankush/anaconda3/envs/argilla_dev_env/lib/python3.9/site-packages/datasets/load.py#1271\1271]8;;\
                             /home/ankush/.cache/huggingface/modules/datasets_modules/datasets/imdb/d6             
                             13c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0 (last                  
                             modified on Thu Dec  7 11:47:14 2023) since it couldn't be found locally              
                             at imdb., or remotely on the Hugging Face Hub.                                        

In [9]:
from argilla import FeedbackDataset, TextField, ResponseSchema, ValueSchema,  SuggestionSchema


# ds = rg.load("imdb_movie_reviews", workspace="test_workspace")abs
# load imdb to Argilla
fds = FeedbackDataset(fields=[TextField(name="text", title="text"), TextField(name="label", title="label")], questions=[
        rg.LabelQuestion(name="label", labels=["positive", "negative"])
])
records = []
for i, record in enumerate(imdb["train"]):
    # print(record)
    label_value = "positive" if record["label"] else "negative"
    record = rg.FeedbackRecord(
        fields={
            "text": record["text"],
            "label": label_value
        },
        responses=[ResponseSchema(values={'label': ValueSchema(value=label_value)})],
        external_id=f"entry-{i}",
    )
    # print(record)
    records.append(record)

fds.add_records(records)

fds.push_to_argilla(name="imdb_movie_reviews", workspace="hello-world-workspace")




Output()

[12/16/23 17:07:49] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=680592;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=379294;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/dataset/local/mixins.py#281\281]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=562365;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=335492;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/dataset/local/mixins.py#282\282]8;;\
                                id=b1c36afb-5903-4846-b3c8-517bb05a7b4e                                            
                                name=imdb_movie_reviews                                                            
                                workspace=Workspace(id=2f16a2ae-f42b-4470-ba95-9079f8f8a384,                       
                             name=hello-world-workspace, inserted_at=2023-12-16 11:11:20.015212,                   
                             updated_at=2023-12-16 11:11:20.015212)                                                
                                url=http://localhost:6900/dataset/b1c36afb-5903-4846-b3c8-517bb05a7b4              
                             e/annotation-mode                                                                     
                                fields=[RemoteTextField(id=UUID('609faf0b-3c15-4a41-9a66-0145c016e5b0              
                             '), client=None, name='text', title='text', required=True, type='text',               
                             use_markdown=False),                                                                  
                             RemoteTextField(id=UUID('5050f2ea-5eee-4308-850b-fe88a8a8867b'),                      
                             client=None, name='label', title='label', required=True, type='text',                 
                             use_markdown=False)]                                                                  
                                questions=[RemoteLabelQuestion(id=UUID('d060ec81-1925-401e-b756-bd85b              
                             f8fe373'), client=None, name='label', title='Label', description=None,                
                             required=True, type='label_selection', labels=['positive', 'negative'],               
                             visible_labels=None)]                                                                 
                                guidelines=None                                                                    
                                metadata_properties=[]                                                             
                             )                                                                                     

RemoteFeedbackDataset(
   id=b1c36afb-5903-4846-b3c8-517bb05a7b4e
   name=imdb_movie_reviews
   workspace=Workspace(id=2f16a2ae-f42b-4470-ba95-9079f8f8a384, name=hello-world-workspace, inserted_at=2023-12-16 11:11:20.015212, updated_at=2023-12-16 11:11:20.015212)
   url=http://localhost:6900/dataset/b1c36afb-5903-4846-b3c8-517bb05a7b4e/annotation-mode
   fields=[RemoteTextField(id=UUID('609faf0b-3c15-4a41-9a66-0145c016e5b0'), client=None, name='text', title='text', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('5050f2ea-5eee-4308-850b-fe88a8a8867b'), client=None, name='label', title='label', required=True, type='text', use_markdown=False)]
   questions=[RemoteLabelQuestion(id=UUID('d060ec81-1925-401e-b756-bd85bf8fe373'), client=None, name='label', title='Label', description=None, required=True, type='label_selection', labels=['positive', 'negative'], visible_labels=None)]
   guidelines=None
   metadata_properties=[]
)

In [24]:
import argilla as rg
from argilla.feedback import TrainingTask, ArgillaTrainer
from datasets import load_dataset
from argilla._constants import DEFAULT_API_KEY
from argilla import FeedbackDataset

dataset = FeedbackDataset.from_argilla(
    name="imdb_movie_reviews",
    workspace="hello-world-workspace"
)
# print(dataset[0])

def formatting_func(sample):
    # print(sample)
    text = sample["text"]
    label = sample["label"]#[0]["value"]
    # print((text, label))
    return(text, label)

# task = TrainingTask.for_text_classification(formatting_func=formatting_func)
task = TrainingTask.for_text_classification(
    text=dataset.field_by_name("text"),
    label=dataset.question_by_name("label"),
)

trainer = ArgillaTrainer(
    dataset=dataset,
    task=task,
    framework="spacy",
    train_size=0.8,
    model="en_core_web_sm",
    # lang="en"
)

trainer.update_config(
    # dev_corpus = "corpora.dev",
    # train_corpus = "corpora.train",
    # seed = 42,
    gpu_allocator = "1",
    # accumulate_gradient = 1,
    # patience = 1600,
    max_epochs = 0,
    max_steps = 2000,
    # eval_frequency = 200,
    # frozen_components = [],
    # annotating_components = [],
    # before_to_disk = None,
    # before_update = None
)

trainer.train(output_dir="new_imdb_output_dir")

[12/16/23 18:19:59] WARNING  WARNING:argilla.client.feedback.training.schemas.base:spaCy `lang` is not  ]8;id=961688;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/training/schemas/base.py\base.py]8;;\:]8;id=614067;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/training/schemas/base.py#245\245]8;;\
                             provided. Using `en`(English) as default language.                                    

[12/16/23 18:20:21] INFO     INFO:ArgillaTrainer:            ArgillaBaseTrainer info:                   ]8;id=869034;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/training/base.py\base.py]8;;\:]8;id=623136;file:///home/ankush/workplace/os_repos/argilla/src/argilla/client/feedback/training/base.py#227\227]8;;\
                                         ______________________________________________________________            
                             ___                                                                                   
                                         These baseline params are fixed:                                          
                                             dataset: RemoteFeedbackDataset(                                       
                                id=b1c36afb-5903-4846-b3c8-517bb05a7b4e                                            
                                name=imdb_movie_reviews                                                            
                                workspace=Workspace(id=2f16a2ae-f42b-4470-ba95-9079f8f8a384,                       
                             name=hello-world-workspace, inserted_at=2023-12-16 11:11:20.015212,                   
                             updated_at=2023-12-16 11:11:20.015212)                                                
                                url=http://localhost:6900/dataset/b1c36afb-5903-4846-b3c8-517bb05a7b4e/            
                             annotation-mode                                                                       
                                fields=[RemoteTextField(id=UUID('609faf0b-3c15-4a41-9a66-0145c016e5b0')            
                             , client=None, name='text', title='text', required=True, type='text',                 
                             use_markdown=False),                                                                  
                             RemoteTextField(id=UUID('5050f2ea-5eee-4308-850b-fe88a8a8867b'),                      
                             client=None, name='label', title='label', required=True, type='text',                 
                             use_markdown=False)]                                                                  
                                questions=[RemoteLabelQuestion(id=UUID('d060ec81-1925-401e-b756-bd85bf8            
                             fe373'), client=None, name='label', title='Label', description=None,                  
                             required=True, type='label_selection', labels=['positive', 'negative'],               
                             visible_labels=None)]                                                                 
                                guidelines=None                                                                    
                                metadata_properties=[]                                                             
                             )                                                                                     
                                             task: formatting_func=None                                            
                             defaults=TextClassificationDefaults(text=RemoteTextField(id=UUID('609faf0b            
                             -3c15-4a41-9a66-0145c016e5b0'), client=None, name='text', title='text',               
                             required=True, type='text', use_markdown=False),                                      
                             label=LabelQuestionUnification(question=RemoteLabelQuestion(id=UUID('d060e            
                             c81-1925-401e-b756-bd85bf8fe373'), client=None, name='label',                         
                             title='Label', description=None, required=True, type='label_selection',               
                             labels=['positive', 'negati

                    WARNING  WARNING:segment:Dictionary values must be serializeable to JSON            ]8;id=999941;file:///home/ankush/anaconda3/envs/argilla_dev_env/lib/python3.9/site-packages/analytics/utils.py\utils.py]8;;\:]8;id=470872;file:///home/ankush/anaconda3/envs/argilla_dev_env/lib/python3.9/site-packages/analytics/utils.py#71\71]8;;\
                             "framework" value spacy of type <enum 'Framework'> is unsupported.                    

                    INFO     INFO:ArgillaTrainer:Updated parameters:                                    ]8;id=833320;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/base.py\base.py]8;;\:]8;id=466695;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/base.py#270\270]8;;\
                             _________________________________________________________________                     
                             WARNING:`ArgillaSpaCyTrainer.update_config` only supports the update of               
                             the `training` arguments defined in the `config.yaml`.                                
                                                                                                                   
                             `ArgillaSpaCyTrainer`                                                                 
                                     dev_corpus: corpora.dev                                                       
                                     train_corpus: corpora.train                                                   
                                     seed: ${system.seed}                                                          
                                     gpu_allocator: 1                                                              
                                     dropout: 0.1                                                                  
                                     accumulate_gradient: 1                                                        
                                     patience: 1600                                                                
                                     max_epochs: 0                                                                 
                                     max_steps: 2000                                                               
                                     eval_frequency: 200                                                           
                                     frozen_components: []                                                         
                                     annotating_components: []                                                     
                                     before_to_disk: None                                                          
                                     before_update: None                                                           

                    WARNING  WARNING:ArgillaSpaCyTrainer:Note that the spaCy training is expected to   ]8;id=224549;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/spacy.py\spacy.py]8;;\:]8;id=971204;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/spacy.py#182\182]8;;\
                             be used through the CLI rather than programmatically, so the dataset                  
                             needs to be dumped into the disk and then loaded from disk. More                      
                             information at https://spacy.io/usage/training#api                                    

                    INFO     INFO:ArgillaSpaCyTrainer:Dumping the train dataset to ./train.spacy       ]8;id=21000;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/spacy.py\spacy.py]8;;\:]8;id=205803;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/spacy.py#188\188]8;;\

[12/16/23 18:20:28] INFO     INFO:ArgillaSpaCyTrainer:Dumping the dev dataset to ./dev.spacy           ]8;id=561457;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/spacy.py\spacy.py]8;;\:]8;id=487449;file:///home/ankush/workplace/os_repos/argilla/src/argilla/training/spacy.py#191\191]8;;\

ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       33.52    0.34
  0     200         53.76       63.08    0.63
  0     400         48.18       76.53    0.77
  0     600         39.91       75.65    0.76


[12/16/23 18:22:28] INFO     INFO:backoff:Backing off send_request(...) for 108.4s                   ]8;id=240393;file:///home/ankush/anaconda3/envs/argilla_dev_env/lib/python3.9/site-packages/backoff/_common.py\_common.py]8;;\:]8;id=733675;file:///home/ankush/anaconda3/envs/argilla_dev_env/lib/python3.9/site-packages/backoff/_common.py#104\104]8;;\
                             (requests.exceptions.ReadTimeout:                                                     
                             HTTPSConnectionPool(host='api.segment.io', port=443): Read timed out.                 
                             (read timeout=15))                                                                    

  0     800         30.78       76.56    0.77
  0    1000         25.84       74.54    0.75
  0    1200         28.70       83.54    0.84
  0    1400         29.52       83.24    0.83
  0    1600         26.72       84.37    0.84
  0    1800         31.33       85.25    0.85
  0    2000         25.51       69.26    0.69


In [20]:
[(doc.text, doc.cats) for doc in list(trainer.predict(["Movie was bad.", " Movie was very good!"]))]

[('Movie was bad.',
  {'positive': 0.3244285583496094, 'negative': 0.6755714416503906}),
 (' Movie was very good!',
  {'positive': 0.6074045300483704, 'negative': 0.392595499753952})]